In [1]:
from flask import Flask, render_template, request

from dataloader import Dataset
from model import Models
from transformers import BertTokenizer
from utils import split_sentence

app = Flask(__name__)

@app.route('/')
def file_upload():
    return render_template('load_file.html')
    
@app.route('/isViolence', methods = ['POST', 'GET'])
def Predict():
    if request.method == 'POST':
        input_text = request.files['파일'].read().decode('utf-8') # 파일 불러오기
        input_text = input_text.split('\r\n')
        
        if input_text == None:
            return render_template('isViolence.html', Output = '')

        sentences = split_sentence(input_text)  # 채팅 목록을 문장 단위로 분리
        # print(sentences)
        result = 0
        for i, sentence in enumerate(sentences[:10000]):
            if (i+1) % 1000 == 0:
                print(f"{i+1}번째 실행 중...")
            result += model.inference(sentence[-1])[1]
        
        ModelOutput = f"해당 채팅방의 폭력성 대화 비율은 {round((result / len(sentences)) * 100, 2)}% 입니다"
        print(ModelOutput)
        return render_template('isViolence.html', Output = ModelOutput)

if __name__ == '__main__':

    train_file_path = './data/train.csv'
    valid_file_path = './data/val.csv'
    test_file_path = './data/test.csv' 
    model_save_dir = './model_checkpoint'       # 모델 저장 디렉토리 경로
    model_load_dir = './model_checkpoint'       # 모델 불러오기 디렉토리 경로

    TRAIN_DATA_LIMIT = 10000

    # # 1. 단일 Dataset 인스턴스 생성 및 데이터 로드
    dataset = Dataset()
    dataset.set_dataset('train', file_path=train_file_path, max_rows=TRAIN_DATA_LIMIT)
    dataset.set_dataset('valid', file_path=valid_file_path)
    dataset.set_dataset('test', file_path=test_file_path)

    # # 2. Models 인스턴스 생성 및 Dataset 연결
    # Models 인스턴스를 만들 때, dataset 객체를 인수로 전달합니다.
    model = Models('koelectra', num_labels = 2, dataset_instance=dataset) 
    
    # # 3. 모델 정의 및 토크나이저 설정
    # model.BERT()를 호출하여 모델을 생성하고, 반환된 토크나이저를 dataset에 설정합니다.
    tokenizer_for_training = model.BERT()  
    dataset.set_tokenizer(tokenizer_for_training)
    
    print(dataset.get_tokenizer())
    
    # # 4. 데이터 로더 생성 (이제 올바른 토크나이저를 사용)
    train, valid, test = dataset.get_dataloader()  
    
    model.about_model()
    model.train(train, valid, epochs = 2, project_title=None) 
    model.test(test)
    model.save_model(model_save_dir)

    # # 5. 추론 및 웹 서버용 모델 로드 (새로운 인스턴스 생성 및 로드)
    # 추론용 모델도 동일한 dataset 인스턴스를 사용하도록 연결
    web_model = Models('koelectra', num_labels = 2, dataset_instance=dataset)
    web_model.load_model(load_dir_path=model_load_dir)
    
    # # 6. 단일 문장 추론 테스트
    sentence, prediction = web_model.inference('이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지')
    print(f"'{sentence}' 은/는 폭력성이 포함된 문장입니다" if prediction == 1 else f"'{sentence}' 은/는 폭력성이 포함되지 않은 문장입니다")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
[train]: 로드 중 파일: ./data/train.csv
[train]: 총 1개의 파일에서 365,500개의 데이터를 성공적으로 병합했습니다.
[train]: 원본 데이터 365,500개를 10,000개로 샘플링합니다.
Train : 10000
[valid]: 로드 중 파일: ./data/val.csv
[valid]: 총 1개의 파일에서 40,612개의 데이터를 성공적으로 병합했습니다.
Valid : 40612
[test]: 로드 중 파일: ./data/test.csv
[test]: 총 1개의 파일에서 44,998개의 데이터를 성공적으로 병합했습니다.
Test : 44998
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
The KOELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddi

/usr/bin/ld: cannot find -lcufile
collect2: error: ld returned 1 exit status


There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Loading model from ./model_checkpoint
'이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지' 은/는 폭력성이 포함된 문장입니다


In [3]:
from flask import Flask, render_template, request

from dataloader import Dataset
from model import Models
from transformers import BertTokenizer
from utils import split_sentence

app = Flask(__name__)

@app.route('/')
def file_upload():
    return render_template('load_file.html')
    
@app.route('/isViolence', methods = ['POST', 'GET'])
def Predict():
    if request.method == 'POST':
        input_text = request.files['파일'].read().decode('utf-8') # 파일 불러오기
        input_text = input_text.split('\r\n')
        
        if input_text == None:
            return render_template('isViolence.html', Output = '')

        sentences = split_sentence(input_text)  # 채팅 목록을 문장 단위로 분리
        # print(sentences)
        result = 0
        for i, sentence in enumerate(sentences[:10000]):
            if (i+1) % 1000 == 0:
                print(f"{i+1}번째 실행 중...")
            result += model.inference(sentence[-1])[1]
        
        ModelOutput = f"해당 채팅방의 폭력성 대화 비율은 {round((result / len(sentences)) * 100, 2)}% 입니다"
        print(ModelOutput)
        return render_template('isViolence.html', Output = ModelOutput)

if __name__ == '__main__':

    train_file_path = './data/train.csv'
    valid_file_path = './data/val.csv'
    test_file_path = './data/test.csv' 
    model_save_dir = './model_checkpoint'       # 모델 저장 디렉토리 경로
    model_load_dir = './model_checkpoint'       # 모델 불러오기 디렉토리 경로

    TRAIN_DATA_LIMIT = 50000

    # # 1. 단일 Dataset 인스턴스 생성 및 데이터 로드
    dataset = Dataset()
    dataset.set_dataset('train', file_path=train_file_path, max_rows=TRAIN_DATA_LIMIT)
    dataset.set_dataset('valid', file_path=valid_file_path)
    dataset.set_dataset('test', file_path=test_file_path)

    # # 2. Models 인스턴스 생성 및 Dataset 연결
    # Models 인스턴스를 만들 때, dataset 객체를 인수로 전달합니다.
    model = Models('koelectra', num_labels = 2, dataset_instance=dataset) 
    
    # # 3. 모델 정의 및 토크나이저 설정
    # model.BERT()를 호출하여 모델을 생성하고, 반환된 토크나이저를 dataset에 설정합니다.
    tokenizer_for_training = model.BERT()  
    dataset.set_tokenizer(tokenizer_for_training)
    
    print(dataset.get_tokenizer())
    
    # # 4. 데이터 로더 생성 (이제 올바른 토크나이저를 사용)
    train, valid, test = dataset.get_dataloader()  
    
    model.about_model()
    model.train(train, valid, epochs = 2, project_title=None) 
    model.test(test)
    model.save_model(model_save_dir)

    # # 5. 추론 및 웹 서버용 모델 로드 (새로운 인스턴스 생성 및 로드)
    # 추론용 모델도 동일한 dataset 인스턴스를 사용하도록 연결
    web_model = Models('koelectra', num_labels = 2, dataset_instance=dataset)
    web_model.load_model(load_dir_path=model_load_dir)
    
    # # 6. 단일 문장 추론 테스트
    sentence, prediction = web_model.inference('이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지')
    print(f"'{sentence}' 은/는 폭력성이 포함된 문장입니다" if prediction == 1 else f"'{sentence}' 은/는 폭력성이 포함되지 않은 문장입니다")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
[train]: 로드 중 파일: ./data/train.csv
[train]: 총 1개의 파일에서 365,500개의 데이터를 성공적으로 병합했습니다.
[train]: 원본 데이터 365,500개를 50,000개로 샘플링합니다.
Train : 50000
[valid]: 로드 중 파일: ./data/val.csv
[valid]: 총 1개의 파일에서 40,612개의 데이터를 성공적으로 병합했습니다.
Valid : 40612
[test]: 로드 중 파일: ./data/test.csv
[test]: 총 1개의 파일에서 44,998개의 데이터를 성공적으로 병합했습니다.
Test : 44998
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
The KOELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddi

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Loading model from ./model_checkpoint
'이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지' 은/는 폭력성이 포함된 문장입니다


In [1]:
from flask import Flask, render_template, request

from dataloader import Dataset
from model import Models
from transformers import BertTokenizer
from utils import split_sentence

app = Flask(__name__)

@app.route('/')
def file_upload():
    return render_template('load_file.html')
    
@app.route('/isViolence', methods = ['POST', 'GET'])
def Predict():
    if request.method == 'POST':
        input_text = request.files['파일'].read().decode('utf-8') # 파일 불러오기
        input_text = input_text.split('\r\n')
        
        if input_text == None:
            return render_template('isViolence.html', Output = '')

        sentences = split_sentence(input_text)  # 채팅 목록을 문장 단위로 분리
        # print(sentences)
        result = 0
        for i, sentence in enumerate(sentences[:10000]):
            if (i+1) % 1000 == 0:
                print(f"{i+1}번째 실행 중...")
            result += model.inference(sentence[-1])[1]
        
        ModelOutput = f"해당 채팅방의 폭력성 대화 비율은 {round((result / len(sentences)) * 100, 2)}% 입니다"
        print(ModelOutput)
        return render_template('isViolence.html', Output = ModelOutput)

if __name__ == '__main__':

    train_file_path = './data/train.csv'
    valid_file_path = './data/val.csv'
    test_file_path = './data/test.csv' 
    model_save_dir = './model_checkpoint'       # 모델 저장 디렉토리 경로
    model_load_dir = './model_checkpoint'       # 모델 불러오기 디렉토리 경로

    TRAIN_DATA_LIMIT = 100000

    # # 1. 단일 Dataset 인스턴스 생성 및 데이터 로드
    dataset = Dataset()
    dataset.set_dataset('train', file_path=train_file_path, max_rows=TRAIN_DATA_LIMIT)
    dataset.set_dataset('valid', file_path=valid_file_path)
    dataset.set_dataset('test', file_path=test_file_path)

    # # 2. Models 인스턴스 생성 및 Dataset 연결
    # Models 인스턴스를 만들 때, dataset 객체를 인수로 전달합니다.
    model = Models('koelectra', num_labels = 2, dataset_instance=dataset) 
    
    # # 3. 모델 정의 및 토크나이저 설정
    # model.BERT()를 호출하여 모델을 생성하고, 반환된 토크나이저를 dataset에 설정합니다.
    tokenizer_for_training = model.BERT()  
    dataset.set_tokenizer(tokenizer_for_training)
    
    print(dataset.get_tokenizer())
    
    # # 4. 데이터 로더 생성 (이제 올바른 토크나이저를 사용)
    train, valid, test = dataset.get_dataloader()  
    
    model.about_model()
    model.train(train, valid, epochs = 2, project_title=None) 
    model.test(test)
    model.save_model(model_save_dir)

    # # 5. 추론 및 웹 서버용 모델 로드 (새로운 인스턴스 생성 및 로드)
    # 추론용 모델도 동일한 dataset 인스턴스를 사용하도록 연결
    web_model = Models('koelectra', num_labels = 2, dataset_instance=dataset)
    web_model.load_model(load_dir_path=model_load_dir)
    
    # # 6. 단일 문장 추론 테스트
    sentence, prediction = web_model.inference('이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지')
    print(f"'{sentence}' 은/는 폭력성이 포함된 문장입니다" if prediction == 1 else f"'{sentence}' 은/는 폭력성이 포함되지 않은 문장입니다")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
[train]: 로드 중 파일: ./data/train.csv
[train]: 총 1개의 파일에서 365,500개의 데이터를 성공적으로 병합했습니다.
[train]: 원본 데이터 365,500개를 100,000개로 샘플링합니다.
Train : 100000
[valid]: 로드 중 파일: ./data/val.csv
[valid]: 총 1개의 파일에서 40,612개의 데이터를 성공적으로 병합했습니다.
Valid : 40612
[test]: 로드 중 파일: ./data/test.csv
[test]: 총 1개의 파일에서 44,998개의 데이터를 성공적으로 병합했습니다.
Test : 44998
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
The KOELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddi

/usr/bin/ld: cannot find -lcufile
collect2: error: ld returned 1 exit status


There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Loading model from ./model_checkpoint
'이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지' 은/는 폭력성이 포함된 문장입니다


In [2]:
from flask import Flask, render_template, request

from dataloader import Dataset
from model import Models
from transformers import BertTokenizer
from utils import split_sentence

app = Flask(__name__)

@app.route('/')
def file_upload():
    return render_template('load_file.html')
    
@app.route('/isViolence', methods = ['POST', 'GET'])
def Predict():
    if request.method == 'POST':
        input_text = request.files['파일'].read().decode('utf-8') # 파일 불러오기
        input_text = input_text.split('\r\n')
        
        if input_text == None:
            return render_template('isViolence.html', Output = '')

        sentences = split_sentence(input_text)  # 채팅 목록을 문장 단위로 분리
        # print(sentences)
        result = 0
        for i, sentence in enumerate(sentences[:10000]):
            if (i+1) % 1000 == 0:
                print(f"{i+1}번째 실행 중...")
            result += model.inference(sentence[-1])[1]
        
        ModelOutput = f"해당 채팅방의 폭력성 대화 비율은 {round((result / len(sentences)) * 100, 2)}% 입니다"
        print(ModelOutput)
        return render_template('isViolence.html', Output = ModelOutput)

if __name__ == '__main__':

    train_file_path = './data/train.csv'
    valid_file_path = './data/val.csv'
    test_file_path = './data/test.csv' 
    model_save_dir = './model_checkpoint'       # 모델 저장 디렉토리 경로
    model_load_dir = './model_checkpoint'       # 모델 불러오기 디렉토리 경로

    TRAIN_DATA_LIMIT = 200000

    # # 1. 단일 Dataset 인스턴스 생성 및 데이터 로드
    dataset = Dataset()
    dataset.set_dataset('train', file_path=train_file_path, max_rows=TRAIN_DATA_LIMIT)
    dataset.set_dataset('valid', file_path=valid_file_path)
    dataset.set_dataset('test', file_path=test_file_path)

    # # 2. Models 인스턴스 생성 및 Dataset 연결
    # Models 인스턴스를 만들 때, dataset 객체를 인수로 전달합니다.
    model = Models('koelectra', num_labels = 2, dataset_instance=dataset) 
    
    # # 3. 모델 정의 및 토크나이저 설정
    # model.BERT()를 호출하여 모델을 생성하고, 반환된 토크나이저를 dataset에 설정합니다.
    tokenizer_for_training = model.BERT()  
    dataset.set_tokenizer(tokenizer_for_training)
    
    print(dataset.get_tokenizer())
    
    # # 4. 데이터 로더 생성 (이제 올바른 토크나이저를 사용)
    train, valid, test = dataset.get_dataloader()  
    
    model.about_model()
    model.train(train, valid, epochs = 2, project_title=None) 
    model.test(test)
    model.save_model(model_save_dir)

    # # 5. 추론 및 웹 서버용 모델 로드 (새로운 인스턴스 생성 및 로드)
    # 추론용 모델도 동일한 dataset 인스턴스를 사용하도록 연결
    web_model = Models('koelectra', num_labels = 2, dataset_instance=dataset)
    web_model.load_model(load_dir_path=model_load_dir)
    
    # # 6. 단일 문장 추론 테스트
    sentence, prediction = web_model.inference('이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지')
    print(f"'{sentence}' 은/는 폭력성이 포함된 문장입니다" if prediction == 1 else f"'{sentence}' 은/는 폭력성이 포함되지 않은 문장입니다")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
[train]: 로드 중 파일: ./data/train.csv
[train]: 총 1개의 파일에서 365,500개의 데이터를 성공적으로 병합했습니다.
[train]: 원본 데이터 365,500개를 200,000개로 샘플링합니다.
Train : 200000
[valid]: 로드 중 파일: ./data/val.csv
[valid]: 총 1개의 파일에서 40,612개의 데이터를 성공적으로 병합했습니다.
Valid : 40612
[test]: 로드 중 파일: ./data/test.csv
[test]: 총 1개의 파일에서 44,998개의 데이터를 성공적으로 병합했습니다.
Test : 44998
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
The KOELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddi

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Loading model from ./model_checkpoint
'이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지' 은/는 폭력성이 포함된 문장입니다


In [3]:
from flask import Flask, render_template, request

from dataloader import Dataset
from model import Models
from transformers import BertTokenizer
from utils import split_sentence

app = Flask(__name__)

@app.route('/')
def file_upload():
    return render_template('load_file.html')
    
@app.route('/isViolence', methods = ['POST', 'GET'])
def Predict():
    if request.method == 'POST':
        input_text = request.files['파일'].read().decode('utf-8') # 파일 불러오기
        input_text = input_text.split('\r\n')
        
        if input_text == None:
            return render_template('isViolence.html', Output = '')

        sentences = split_sentence(input_text)  # 채팅 목록을 문장 단위로 분리
        # print(sentences)
        result = 0
        for i, sentence in enumerate(sentences[:10000]):
            if (i+1) % 1000 == 0:
                print(f"{i+1}번째 실행 중...")
            result += model.inference(sentence[-1])[1]
        
        ModelOutput = f"해당 채팅방의 폭력성 대화 비율은 {round((result / len(sentences)) * 100, 2)}% 입니다"
        print(ModelOutput)
        return render_template('isViolence.html', Output = ModelOutput)

if __name__ == '__main__':

    train_file_path = './data/train.csv'
    valid_file_path = './data/val.csv'
    test_file_path = './data/test.csv' 
    model_save_dir = './model_checkpoint'       # 모델 저장 디렉토리 경로
    model_load_dir = './model_checkpoint'       # 모델 불러오기 디렉토리 경로

    TRAIN_DATA_LIMIT = 365500

    # # 1. 단일 Dataset 인스턴스 생성 및 데이터 로드
    dataset = Dataset()
    dataset.set_dataset('train', file_path=train_file_path, max_rows=TRAIN_DATA_LIMIT)
    dataset.set_dataset('valid', file_path=valid_file_path)
    dataset.set_dataset('test', file_path=test_file_path)

    # # 2. Models 인스턴스 생성 및 Dataset 연결
    # Models 인스턴스를 만들 때, dataset 객체를 인수로 전달합니다.
    model = Models('koelectra', num_labels = 2, dataset_instance=dataset) 
    
    # # 3. 모델 정의 및 토크나이저 설정
    # model.BERT()를 호출하여 모델을 생성하고, 반환된 토크나이저를 dataset에 설정합니다.
    tokenizer_for_training = model.BERT()  
    dataset.set_tokenizer(tokenizer_for_training)
    
    print(dataset.get_tokenizer())
    
    # # 4. 데이터 로더 생성 (이제 올바른 토크나이저를 사용)
    train, valid, test = dataset.get_dataloader()  
    
    model.about_model()
    model.train(train, valid, epochs = 2, project_title=None) 
    model.test(test)
    model.save_model(model_save_dir)

    # # 5. 추론 및 웹 서버용 모델 로드 (새로운 인스턴스 생성 및 로드)
    # 추론용 모델도 동일한 dataset 인스턴스를 사용하도록 연결
    web_model = Models('koelectra', num_labels = 2, dataset_instance=dataset)
    web_model.load_model(load_dir_path=model_load_dir)
    
    # # 6. 단일 문장 추론 테스트
    sentence, prediction = web_model.inference('이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지')
    print(f"'{sentence}' 은/는 폭력성이 포함된 문장입니다" if prediction == 1 else f"'{sentence}' 은/는 폭력성이 포함되지 않은 문장입니다")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
[train]: 로드 중 파일: ./data/train.csv
[train]: 총 1개의 파일에서 365,500개의 데이터를 성공적으로 병합했습니다.
Train : 365500
[valid]: 로드 중 파일: ./data/val.csv
[valid]: 총 1개의 파일에서 40,612개의 데이터를 성공적으로 병합했습니다.
Valid : 40612
[test]: 로드 중 파일: ./data/test.csv
[test]: 총 1개의 파일에서 44,998개의 데이터를 성공적으로 병합했습니다.
Test : 44998
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
The KOELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddi

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Loading model from ./model_checkpoint
'이런건 걍 돈없고 멍청한 전라도 여자들 겨냥한 상술이지' 은/는 폭력성이 포함된 문장입니다
